In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

In [2]:
# Read the data file
da = pd.read_csv("nhanes_2015_2016.csv")

# Drop unused columns, drop rows with any missing values.
vars = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI",
        "SMQ020", "SDMVSTRA", "SDMVPSU"]
da = da[vars].dropna()

In [3]:
da["group"] = 10*da.SDMVSTRA + da.SDMVPSU

In [4]:
model = sm.GEE.from_formula("BPXSY1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.030


In [5]:
# Recode smoking to a simple binary variable
da["smq"] = da.SMQ020.replace({2: 0, 7: np.nan, 9: np.nan})

for v in ["BPXSY1", "RIDAGEYR", "BMXBMI", "smq", "SDMVSTRA"]:
    model = sm.GEE.from_formula(v + " ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
    result = model.fit()
    print(v, result.cov_struct.summary())

BPXSY1 The correlation between two observations in the same cluster is 0.030
RIDAGEYR The correlation between two observations in the same cluster is 0.035
BMXBMI The correlation between two observations in the same cluster is 0.039
smq The correlation between two observations in the same cluster is 0.026
SDMVSTRA The correlation between two observations in the same cluster is 0.959


In [6]:
for k in range(10):
    da["noise"] = np.random.normal(size=da.shape[0])
    model = sm.GEE.from_formula("noise ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
    result = model.fit()
    print(v, result.cov_struct.summary())

SDMVSTRA The correlation between two observations in the same cluster is -0.000
SDMVSTRA The correlation between two observations in the same cluster is -0.001
SDMVSTRA The correlation between two observations in the same cluster is -0.001
SDMVSTRA The correlation between two observations in the same cluster is -0.001
SDMVSTRA The correlation between two observations in the same cluster is -0.001
SDMVSTRA The correlation between two observations in the same cluster is 0.001
SDMVSTRA The correlation between two observations in the same cluster is 0.001
SDMVSTRA The correlation between two observations in the same cluster is 0.000
SDMVSTRA The correlation between two observations in the same cluster is -0.001
SDMVSTRA The correlation between two observations in the same cluster is 0.001


In [7]:
model = sm.GEE.from_formula("BPXSY1 ~ RIDAGEYR", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.019


In [8]:
# Create a labeled version of the gender variable
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

model = sm.GEE.from_formula("BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)",
           groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.013


In [9]:
# Fit a linear model with OLS
model1 = sm.OLS.from_formula("BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)",
           data=da)
result1 = model1.fit()

# Fit a marginal linear model using GEE to handle dependent data
model2 = sm.GEE.from_formula("BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)",
           groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result2 = model2.fit()

x = pd.DataFrame({"OLS_params": result1.params, "OLS_SE": result1.bse,
                  "GEE_params": result2.params, "GEE_SE": result2.bse})
x = x[["OLS_params", "OLS_SE", "GEE_params", "GEE_SE"]]
print(x)

                   OLS_params    OLS_SE  GEE_params    GEE_SE
Intercept           91.736583  1.339378   92.168530  1.384309
RIAGENDRx[T.Male]    3.671294  0.453763    3.650245  0.454498
C(RIDRETH1)[T.2]     0.855488  0.819486    0.159296  0.767025
C(RIDRETH1)[T.3]    -1.796132  0.671954   -2.233280  0.760228
C(RIDRETH1)[T.4]     3.813314  0.732355    3.105654  0.881580
C(RIDRETH1)[T.5]    -0.455347  0.808948   -0.439831  0.813675
RIDAGEYR             0.478699  0.012901    0.474101  0.018493
BMXBMI               0.278015  0.033285    0.280205  0.038553


In [10]:
# Relabel the levels, convert rare categories to missing.
da["DMDEDUC2x"] = da.DMDEDUC2.replace({1: "lt9", 2: "x9_11", 3: "HS", 4: "SomeCollege",
                                       5: "College", 7: np.nan, 9: np.nan})

# Fit a basic GLM
model1 = sm.GLM.from_formula("smq ~ RIDAGEYR + RIAGENDRx + C(DMDEDUC2x)",
           family=sm.families.Binomial(), data=da)
result1 = model1.fit()
result1.summary()

# Fit a marginal GLM using GEE
model2 = sm.GEE.from_formula("smq ~ RIDAGEYR + RIAGENDRx + C(DMDEDUC2x)",
           groups="group", family=sm.families.Binomial(),
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result2 = model2.fit(start_params=result1.params)

x = pd.DataFrame({"OLS_params": result1.params, "OLS_SE": result1.bse,
                  "GEE_params": result2.params, "GEE_SE": result2.bse})
x = x[["OLS_params", "OLS_SE", "GEE_params", "GEE_SE"]]
print(x)

                             OLS_params    OLS_SE  GEE_params    GEE_SE
Intercept                     -2.305999  0.114308   -2.249820  0.140567
RIAGENDRx[T.Male]              0.909597  0.060167    0.908682  0.062342
C(DMDEDUC2x)[T.HS]             0.943364  0.089663    0.887965  0.095397
C(DMDEDUC2x)[T.SomeCollege]    0.832227  0.084361    0.771636  0.104449
C(DMDEDUC2x)[T.lt9]            0.266228  0.109183    0.321784  0.141327
C(DMDEDUC2x)[T.x9_11]          1.098561  0.106697    1.062149  0.138401
RIDAGEYR                       0.018257  0.001725    0.017416  0.001803


In [11]:
# Fit a multilevel (mixed effects) model to handle dependent data
model = sm.MixedLM.from_formula("BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)",
           groups="group", data=da)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
           Mixed Linear Model Regression Results
============================================================
Model:             MixedLM  Dependent Variable:  BPXSY1     
No. Observations:  5102     Method:              REML       
No. Groups:        30       Scale:               256.6952   
Min. group size:   106      Log-Likelihood:      -21409.8702
Max. group size:   226      Converged:           Yes        
Mean group size:   170.1                                    
------------------------------------------------------------
                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------
Intercept         92.173    1.402 65.752 0.000 89.426 94.921
RIAGENDRx[T.Male]  3.650    0.452  8.084 0.000  2.765  4.535
C(RIDRETH1)[T.2]   0.153    0.887  0.172 0.863 -1.586  1.891
C(RIDRETH1)[T.3]  -2.238    0.758 -2.954 0.003 -3.723 -0.753
C(RIDRETH1)[T.4]   3.098    0.836  3.707 0.000  1.460  4.737
C(RIDRETH1)[T.5]  -0.439    0.878 -0.500 0.617 -2.161  1.282
RIDAGEYR           0.474    0.013 36.482 0.000  0.449  0.500
BMXBMI             0.280    0.033  8.404 0.000  0.215  0.346
group Var          3.615    0.085                           
============================================================

"""

In [12]:
result.random_effects

{1191: group   -1.630976
 dtype: float64,
 1192: group   -0.086162
 dtype: float64,
 1201: group   -2.042661
 dtype: float64,
 1202: group   -0.147472
 dtype: float64,
 1211: group    0.280623
 dtype: float64,
 1212: group    1.580732
 dtype: float64,
 1221: group    0.283347
 dtype: float64,
 1222: group    0.131512
 dtype: float64,
 1231: group   -2.038171
 dtype: float64,
 1232: group    0.617651
 dtype: float64,
 1241: group    2.878488
 dtype: float64,
 1242: group   -0.519364
 dtype: float64,
 1251: group    2.064967
 dtype: float64,
 1252: group    1.521281
 dtype: float64,
 1261: group   -1.261975
 dtype: float64,
 1262: group    0.980846
 dtype: float64,
 1271: group    0.118031
 dtype: float64,
 1272: group   -0.128397
 dtype: float64,
 1281: group   -0.384862
 dtype: float64,
 1282: group   -3.582111
 dtype: float64,
 1291: group   -3.271017
 dtype: float64,
 1292: group   -0.829538
 dtype: float64,
 1301: group   -0.884171
 dtype: float64,
 1302: group    2.790657
 dtype: f

In [13]:
da["age_cen"] = da.groupby("group").RIDAGEYR.transform(lambda x: x - x.mean())

model = sm.MixedLM.from_formula("BPXSY1 ~ age_cen + RIAGENDRx + BMXBMI + C(RIDRETH1)",
           groups="group", vc_formula={"age_cen": "0+age_cen"}, data=da)
result = model.fit()
result.summary()

c:\users\neel\appdata\local\programs\python\python38\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2149: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
             Mixed Linear Model Regression Results
===============================================================
Model:              MixedLM   Dependent Variable:   BPXSY1     
No. Observations:   5102      Method:               REML       
No. Groups:         30        Scale:                263.7323   
Min. group size:    106       Log-Likelihood:       -21469.9240
Max. group size:    226       Converged:            Yes        
Mean group size:    170.1                                      
---------------------------------------------------------------
                   Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
---------------------------------------------------------------
Intercept         115.207    1.209 95.265 0.000 112.836 117.577
RIAGENDRx[T.Male]   3.643    0.457  7.962 0.000   2.746   4.539
C(RIDRETH1)[T.2]    1.167    0.827  1.412 0.158  -0.453   2.787
C(RIDRETH1)[T.3]   -1.659    0.679 -2.444 0.015  -2.989  -0.328
C(RIDRETH1)[T.4]    3.610    0.739  4.884 0.000   2.161   5.058
C(RIDRETH1)[T.5]   -1.208    0.816 -1.480 0.139  -2.807   0.392
age_cen             0.467    0.018 26.235 0.000   0.432   0.502
BMXBMI              0.288    0.034  8.574 0.000   0.222   0.353
age_cen Var         0.004    0.000                             
===============================================================

"""

In [14]:
model = sm.MixedLM.from_formula("BPXSY1 ~ age_cen + RIAGENDRx + BMXBMI + C(RIDRETH1)",
           groups="group", re_formula="1+age_cen", data=da)
result = model.fit()
result.summary()

c:\users\neel\appdata\local\programs\python\python38\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2149: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
              Mixed Linear Model Regression Results
=================================================================
Model:                MixedLM   Dependent Variable:   BPXSY1     
No. Observations:     5102      Method:               REML       
No. Groups:           30        Scale:                255.4451   
Min. group size:      106       Log-Likelihood:       -21413.6193
Max. group size:      226       Converged:            Yes        
Mean group size:      170.1                                      
-----------------------------------------------------------------
                     Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
-----------------------------------------------------------------
Intercept           115.467    1.340 86.173 0.000 112.840 118.093
RIAGENDRx[T.Male]     3.662    0.451  8.121 0.000   2.778   4.546
C(RIDRETH1)[T.2]      0.023    0.898  0.025 0.980  -1.738   1.783
C(RIDRETH1)[T.3]     -2.251    0.778 -2.893 0.004  -3.775  -0.726
C(RIDRETH1)[T.4]      3.011    0.854  3.524 0.000   1.336   4.686
C(RIDRETH1)[T.5]     -0.585    0.893 -0.655 0.512  -2.336   1.165
age_cen               0.466    0.018 26.286 0.000   0.431   0.501
BMXBMI                0.283    0.033  8.497 0.000   0.218   0.349
group Var             8.655    0.169                             
group x age_cen Cov   0.119    0.004                             
age_cen Var           0.004    0.000                             
=================================================================

"""